In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.mongodb.aio import AsyncMongoDBSaver, AsyncIOMotorClient
from mentoragents.workflow.graph import MentorGraph 
from mentoragents.core.config import settings 
from mentoragents.models.mentor import Mentor

OPIK: ========================
The API key must be specified to log data to https://www.comet.com/opik.
You can use `opik configure` CLI command to configure your environment for logging.
See the configuration details in the docs: https://www.comet.com/docs/opik/tracing/sdk_configuration.




2025-06-30 18:44:46.751 | WARNING  | mentoragents.workflow.prompt:__init__:17 - Can't use Opik to version the prompt (probably due to missing or invalid credentials). Falling back to local prompt. The prompt is not versioned, but it's still usable.
2025-06-30 18:44:46.996 | WARNING  | mentoragents.workflow.prompt:__init__:17 - Can't use Opik to version the prompt (probably due to missing or invalid credentials). Falling back to local prompt. The prompt is not versioned, but it's still usable.
2025-06-30 18:44:47.214 | WARNING  | mentoragents.workflow.prompt:__init__:17 - Can't use Opik to version the prompt (probably due to missing or invalid credentials). Falling back to local prompt. The prompt is not versioned, but it's still usable.
2025-06-30 18:44:47.447 | WARNING  | mentoragents.workflow.prompt:__init__:17 - Can't use Opik to version the prompt (probably due to missing or invalid credentials). Falling back to local prompt. The prompt is not versioned, but it's still usable.
2025

In [3]:
async def generate_response_without_memory(mentor : Mentor, messages : list): 
    graph = MentorGraph().build()
    graph = graph.compile()
    output_state = await graph.ainvoke(
        input = {
            "messages" : messages, 
            "mentor_name" : mentor.mentor_name,
            "mentor_expertise" : mentor.mentor_expertise,
            "mentor_perspective" : mentor.mentor_perspective,
            "mentor_style" : mentor.mentor_style,   
        }
    )

    last_message = output_state["messages"][-1]
    return last_message 

async def generate_response_with_memory(mentor : Mentor, messages : list):
    async with AsyncMongoDBSaver.from_conn_string(
        conn_string = settings.MONGO_URI,
        db_name = settings.MONGO_DB_NAME,
        checkpoint_collection_name = settings.MONGO_STATE_CHECKPOINT_COLLECTION,
        writes_collection_name = settings.MONGO_STATE_WRITES_COLLECTION
    ) as checkpoint_storage:
        graph = MentorGraph().build()
        graph = graph.compile(checkpointer = checkpoint_storage)

        config = {
            "configurable" : {
                "thread_id" : mentor.id,
            }
        }

        output_state = await graph.ainvoke(
            input = {
                "messages" : messages,
                "mentor_name" : mentor.mentor_name,
                "mentor_expertise" : mentor.mentor_expertise,
                "mentor_perspective" : mentor.mentor_perspective,
                "mentor_style" : mentor.mentor_style,
            },
            config = config
        )

        last_message = output_state["messages"][-1]
        return last_message 

### MentorAgent without short_term memory

In [4]:
test_mentor = Mentor(
    id = "andrej_karpathy",
    mentor_name = "Andrej Karpathy",
    mentor_expertise = "AI and Machine Learning",
    mentor_perspective = "AI and Machine Learning",
    mentor_style = "AI and Machine Learning",
)

user_message = [
    HumanMessage(content="Hello, my name is Shubham")
]

await generate_response_without_memory(test_mentor, user_message)

2025-06-30 18:44:48.716 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
/Users/shubhamp/Documents/new/mentoragents/backend/venvmentor/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-30 18:45:04.035 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:25 - Initializing MongoDBAtlasVectorSearchRetriever with embedding model: sentence-transformers/all-MiniLM-L6-v2 and k: 3
2025-06-30 18:45:04.208 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
2025-06-30 18:45:06.867 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:

AIMessage(content='Hi Shubham, I’m Andrej Karpathy. I focus on AI and machine learning. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 274, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bo8dg6ejJjogsZpMTjLbDM5ruaGag', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0213301b-22cd-4fff-95bb-588c60984ec6-0', usage_metadata={'input_tokens': 274, 'output_tokens': 28, 'total_tokens': 302, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
messages = [
    HumanMessage(content="Do you know my name?")
]

await generate_response_without_memory(test_mentor, messages)

2025-06-30 18:45:08.812 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
2025-06-30 18:45:11.674 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:25 - Initializing MongoDBAtlasVectorSearchRetriever with embedding model: sentence-transformers/all-MiniLM-L6-v2 and k: 3
2025-06-30 18:45:11.779 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
2025-06-30 18:45:14.738 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:25 - Initializing MongoDBAtlasVectorSearchRetriever with embedding model: sentence-transformers/all-MiniLM-L6-v2 and k: 3


AIMessage(content="I don't know your name yet. What's your name?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 272, 'total_tokens': 284, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bo8dnOxRxqHaeiQtXLGIl7HyA55Wo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--33aaafb4-6e75-41c2-ae04-b1982b8b5e28-0', usage_metadata={'input_tokens': 272, 'output_tokens': 12, 'total_tokens': 284, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### PhiloAgent with short term memory


In [6]:
test_mentor = Mentor(
    id = "andrej_karpathy",
    mentor_name = "Andrej Karpathy",
    mentor_expertise = "AI and Machine Learning",
    mentor_perspective = "AI and Machine Learning",
    mentor_style = "AI and Machine Learning",
)

user_message = [
    HumanMessage(content="Hello, my name is Shubham")
]

await generate_response_with_memory(test_mentor, user_message)

2025-06-30 18:45:16.835 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
2025-06-30 18:45:19.648 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:25 - Initializing MongoDBAtlasVectorSearchRetriever with embedding model: sentence-transformers/all-MiniLM-L6-v2 and k: 3
2025-06-30 18:45:19.738 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
2025-06-30 18:45:22.471 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:25 - Initializing MongoDBAtlasVectorSearchRetriever with embedding model: sentence-transformers/all-MiniLM-L6-v2 and k: 3


AIMessage(content="Hi Shubham, nice to meet you! I'm Andrej Karpathy. What can I help you with today regarding AI or machine learning?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 354, 'total_tokens': 384, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bo8dv81UQViq7VphdG9r2ajf1KVec', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--dbbc4605-6040-4b93-8802-6a092d7a7477-0', usage_metadata={'input_tokens': 354, 'output_tokens': 30, 'total_tokens': 384, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
messages = [
    HumanMessage(content="Do you know my name?")
]

await generate_response_with_memory(test_mentor, messages)

2025-06-30 18:45:25.293 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
2025-06-30 18:45:28.318 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:25 - Initializing MongoDBAtlasVectorSearchRetriever with embedding model: sentence-transformers/all-MiniLM-L6-v2 and k: 3
2025-06-30 18:45:28.414 | INFO     | mentoragents.rag.embeddings:get_hf_model:19 - Initializing HuggingFaceEmbeddings with model name: sentence-transformers/all-MiniLM-L6-v2 and device: cpu
2025-06-30 18:45:31.267 | INFO     | mentoragents.rag.retrievers:get_hybrid_search_mongodb_retriever:25 - Initializing MongoDBAtlasVectorSearchRetriever with embedding model: sentence-transformers/all-MiniLM-L6-v2 and k: 3


AIMessage(content='Yes, your name is Shubham. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 397, 'total_tokens': 414, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bo8e4z7P45tjvrfCjZrh24nSmX6d2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3461e65-46cf-48a6-b8e3-0447d26657e4-0', usage_metadata={'input_tokens': 397, 'output_tokens': 17, 'total_tokens': 414, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})